In [4]:
import json

In [5]:
config = {
    "default": "Cas9_geCRISPR_CatBoost_Peng_offtarget",
    "on-target report": "templates/ontarget.template.html",
    "off-target report": "templates/offtarget.template.html",
    "Cas9_geCRISPR_CatBoost_Peng_offtarget": {
        "on": "./models/test/revcore_full_model_2.ptch",
        "off": "./models/test/peng_difference.ptch",
        "on_reg": "./models/test/gecrispr.cat",
        "umap": "./models/test/test_umap.bin",
        "reg_type": "catboost",
        "guide_length": 20,
        "use_pam": False,
        "pam_before": False,
        "PAM": "[ATGC]GG",
        "background": "./models/test/test_background.pkl",
        "max": 0.048707843,
        "min": 0.22760779
    },
    "off_target_indices": ["indices/ecoli.pkl"],
    "k_neighbors": 10,
    "offtarget_batch_size": 102400
}

In [6]:
with open("config.json", "w") as oh:
    json.dump(config, oh)